In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel
from langchain_tavily import TavilySearch

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver


ModuleNotFoundError: No module named 'langgraph.checkpoint.sqlite'

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

from autogen import ConversableAgent


In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.tools import DuckDuckGoSearchRun
import os

# 2. Initialize DuckDuckGo Search tool
duckduckgo_search = DuckDuckGoSearchRun()

# LLM definition
llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="deepseek/deepseek-r1-0528:free"
)

# Prompt for step-by-step reasoning
prompt = PromptTemplate(
    template="Question: {question}\nAnswer: Let's think step by step.",
    input_variables=["question"]
)


In [19]:
from langchain.schema import HumanMessage
def ask_handler(state):
    # You can get input from the user, or set programmatically.
    question = state.get("question") or input("Your question: ")
    state["question"] = question
    return state

def search_handler(state):
    # Replace with a real web search tool, e.g., using SerpAPI
    question = state["question"]
    # For demo, just echo or implement your own retrieval logic
    search_results = duckduckgo_search.run(tool_input=question)
    state["search_results"] = search_results
    return state

def summarize_handler(state):
    search_results = state["search_results"]
    prompt_text = prompt.format_prompt(question="Summarize the following:\n" + search_results).to_string()
    response = llm([HumanMessage(content=prompt_text)])
    summary = response.content
    state["summary"] = summary
    return state

def respond_handler(state):
    # Final synthesis (can add additional reasoning)
    response = f"Research complete. Summary: {state.get('summary', 'No summary generated.')}"
    state["response"] = response
    return state


In [20]:
from langgraph.graph.state import StateGraph

# Define empty dict as the state schema for simplicity, or a dataclass for a schema
graph = StateGraph(state_schema=dict)

graph.add_node("ask", ask_handler)
graph.add_node("search", search_handler)
graph.add_node("summarize", summarize_handler)
graph.add_node("respond", respond_handler)

graph.set_entry_point("ask")
graph.add_edge("ask", "search")
graph.add_edge("search", "summarize")
graph.add_edge("summarize", "respond")

research_agent = graph.compile()


In [21]:
state = {}  # or pre-populated with {"question": "Your research topic"}
final_state = research_agent.invoke(state)
print(final_state.get("response"))


/home/ajeet/miniconda3/envs/ai-agents/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Research complete. Summary: Here's a concise summary of the key points:

1.  **Data Science Definition & Composition:** Data science is an interdisciplinary field combining **statistics, computer science, AND domain-specific knowledge** to extract insights from data. It's more complex than just "stat + CS".
2.  **Academic Landscape:** Dedicated "Data Science" departments are still relatively uncommon internationally, though they might be more prevalent as trends in some regions (like China).
3.  **Career Paths & Entry Points:**
    *   **Data Analyst:** Often serves as an entry-level role (especially for new graduates), typically within IT departments, acting as a stepping stone to become a Data Scientist or Data Engineer.
    *   **Data Scientist/Engineer:** Represent more senior or specialized technical roles.
4.  **Author's Career Dilemma:** The author (with an undergrad in Marketing) is considering a Master's in Business Analytics (BA) or Data Science in Management primarily becaus

/home/ajeet/miniconda3/envs/ai-agents/lib/python3.13/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
